In [1]:
import pandas as pd
import os

# Import docx
import docx
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.table import WD_ALIGN_VERTICAL
from docx import Document
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from docx.shared import Cm
from docx.shared import Pt
from docx.shared import RGBColor

# Import QRCode from pyqrcode 
import pyqrcode 
import png 
from pyqrcode import QRCode

# Import Pillow
from PIL import Image, ImageOps
from PIL import ImageFont
from PIL import ImageDraw 

In [2]:
def set_repeat_table_header(row):
    """ set repeat table row on every new page
    """
    tr = row._tr
    trPr = tr.get_or_add_trPr()
    tblHeader = OxmlElement('w:tblHeader')
    tblHeader.set(qn('w:val'), "true")
    trPr.append(tblHeader)
    return row

def tight_margin(doc):

    current_section = doc.sections[-1]
    current_section.top_margin = Cm(1.4)
    current_section.bottom_margin = Cm(1.4)
    current_section.left_margin = Cm(2)
    current_section.right_margin = Cm(2)
    
def tight_margin2(doc):

    current_section = doc.sections[-1]
    current_section.top_margin = Cm(1.3)
    current_section.bottom_margin = Cm(1.3)
    current_section.left_margin = Cm(2)
    current_section.right_margin = Cm(2)
    
def tight_margin3(doc):

    current_section = doc.sections[-1]
    current_section.top_margin = Cm(1.3)
    current_section.bottom_margin = Cm(1.3)
    current_section.left_margin = Cm(1.3)
    current_section.right_margin = Cm(1.3)

In [3]:
def generate_qr_code(directory, s):
    
    # Generate the QR code 
    url = pyqrcode.create(s)
    
    # Generate image filename
    filename = os.path.join(directory, s+".png")
    
    # Create and save the png file naming "myqr.png" 
    url.png(filename, scale = 3)
    
    img = Image.open(filename)
    rgbimg = img.convert('RGB')
    W, H = rgbimg.size
    draw = ImageDraw.Draw(rgbimg)
    # Add text
    msg = "{}".format(s)
    font = ImageFont.truetype("FONTS/arialbd.ttf", 11)
    w, h = draw.textsize(msg, font=font)
    draw.text(((W+1-w)/2,H-11), msg, font=font, fill="#00c1e4")
    rgbimg.save(filename)

## Template with 4 codes / row (2 for each participant)

In [4]:
def generate_qr_codes_2prow(directory, df, n0, n1):
    
    for _, row in df.iterrows():
        
        f = row['name']
        fname = row['label']
        
        word_doc = 'TIMCI Tanzania - F{0:04d} {1} - Participants {2} to {3}.docx'.format(f, fname, n0, n1)
        
        # Create a new Word document
        doc = Document()
        doc.styles['Normal'].font.name = "Segoe UI"
        doc.styles['Normal'].font.size = Pt(10)
        tight_margin(doc)
        
        # add a table to the end and create a reference variable
        # extra row is so we can add the header row
        tab = doc.add_table(rows=int((n1-n0+1)/2)+1, cols=4)
        set_repeat_table_header(tab.rows[0])

        # add the header rows.
        tab.cell(0,0).text = "Stika kwa mshiriki"
        tab.cell(0,0).vertical_alignment = WD_ALIGN_VERTICAL.CENTER
        paragraph = tab.cell(0,0).paragraphs[0]
        paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
        tab.cell(0,1).text = "Stika itakayotunzwa katika faili la mradi"
        tab.cell(0,1).vertical_alignment = WD_ALIGN_VERTICAL.CENTER
        paragraph = tab.cell(0,1).paragraphs[0]
        paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
        tab.cell(0,2).text = "Stika kwa mshiriki"
        tab.cell(0,2).vertical_alignment = WD_ALIGN_VERTICAL.CENTER
        paragraph = tab.cell(0,2).paragraphs[0]
        paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
        tab.cell(0,3).text = "Stika itakayotunzwa katika faili la mradi"
        tab.cell(0,3).vertical_alignment = WD_ALIGN_VERTICAL.CENTER
        paragraph = tab.cell(0,3).paragraphs[0]
        paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

        # add rows
        for p in range(n0-n0+1, int((n1-n0+1)/2)+1):

            # String which represents the QR code 
            s1 = "T-F{0:04d}-P{1:04d}".format(f, n0-1+2*p-1)

            # Generate QR code 
            url = pyqrcode.create(s1)

            # Create and save the png file naming "myqr.png" 
            url.png(os.path.join(directory, s1+".png"), scale = 2)
            
            infile1 = os.path.join(os.path.join(directory, s1+".png"))
            
            paragraph = tab.cell(p,0).paragraphs[0]
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            run = paragraph.add_run()
            pic = run.add_picture(infile1)
            paragraph.paragraph_format.space_after = Pt(0)
            paragraph = tab.cell(p,0).add_paragraph(s1)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            paragraph.paragraph_format.space_after = Pt(3)
            paragraph = tab.cell(p,0).add_paragraph("Date: ____ / ____ / _______")
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            
            paragraph = tab.cell(p,1).paragraphs[0]
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            run = paragraph.add_run()
            pic = run.add_picture(infile1)
            paragraph.paragraph_format.space_after = Pt(0)
            paragraph = tab.cell(p,1).add_paragraph(s1)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            paragraph.paragraph_format.space_after = Pt(3)
            paragraph = tab.cell(p,1).add_paragraph("Date: ____ / ____ / _______")
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            
            # String which represents the QR code 
            s2 = "T-F{0:04d}-P{1:04d}".format(f, n0-1+2*p)

            # Generate QR code 
            url = pyqrcode.create(s2)

            # Create and save the png file naming "myqr.png" 
            url.png(os.path.join(directory, s2+".png"), scale = 2)
            
            infile2 = os.path.join(os.path.join(directory, s2+".png"))
            
            paragraph = tab.cell(p,2).paragraphs[0]
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            run = paragraph.add_run()
            pic = run.add_picture(infile2)
            paragraph.paragraph_format.space_after = Pt(0)
            paragraph = tab.cell(p,2).add_paragraph(s2)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            paragraph.paragraph_format.space_after = Pt(3)
            paragraph = tab.cell(p,2).add_paragraph("Date: ____ / ____ / _______")
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            
            paragraph = tab.cell(p,3).paragraphs[0]
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            run = paragraph.add_run()
            pic = run.add_picture(infile2)
            paragraph.paragraph_format.space_after = Pt(0)
            paragraph = tab.cell(p,3).add_paragraph(s2)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            paragraph.paragraph_format.space_after = Pt(3)
            paragraph = tab.cell(p,3).add_paragraph("Date: ____ / ____ / _______")
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            
        # save the doc
        doc.save(os.path.join(directory, word_doc))

## All templates

In [5]:
def generate_tz_qr_codes(directory, df, n0, n1):
    
    for _, row in df.iterrows():
        
        f = row['name']
        fname = row['label']
        
        word_doc = 'TIMCI Tanzania - F{0:04d} {1} - Participants {2} to {3}.docx'.format(f, fname, n0, n1)
        
        # Create a new Word document
        doc = Document()
        doc.styles['Normal'].font.name = "Segoe UI"
        doc.styles['Normal'].font.size = Pt(10)
        tight_margin(doc)
        
        # add a table to the end and create a reference variable
        # extra row is so we can add the header row
        tab = doc.add_table(rows=n1-n0+2, cols=4)
        set_repeat_table_header(tab.rows[0])

        # add the header rows
        tab.cell(0,0).merge(tab.cell(0,1))
        tab.cell(0,0).text = "Sticker given to the participant"
        paragraph = tab.cell(0,0).paragraphs[0]
        paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
        
        tab.cell(0,2).merge(tab.cell(0,3))
        tab.cell(0,2).text = "Sticker to be placed on the signed ICF kept in the site study file"
        paragraph = tab.cell(0,2).paragraphs[0]
        paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

        # add rows
        for p in range(n0-n0+1, int((n1-n0+1)/2)+1):
            
            # IHI header
            tab.cell(2*p-1,0).merge(tab.cell(2*p-1,1))
            ccell = tab.cell(2*p-1,0)
            paragraph = ccell.paragraphs[0]
            run = paragraph.add_run()
            pic = run.add_picture(r"C:\Users\langhe\Pictures\logo\ihi2.png")
            pic.width = int(pic.width/2)
            pic.height = int(pic.height/2)

            # String which represents the QR code 
            s1 = "T-F{0:04d}-P{1:04d}".format(f, n0-1+2*p-1)

            # Generate QR code 
            url = pyqrcode.create(s1)

            # Create and save the png file naming "myqr.png" 
            url.png(os.path.join(directory, s1+".png"), scale = 2)
            infile1 = os.path.join(os.path.join(directory, s1+".png"))
            
            ccell = tab.cell(2*p,0)
            paragraph = ccell.paragraphs[0]
            run = paragraph.add_run()
            run.add_text("TIMCI")
            run.bold = True
            font = run.font
            font.size = Pt(9)
            font.color.rgb = RGBColor(125,187,214)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
            paragraph.paragraph_format.space_after = Pt(0)
            paragraph = ccell.add_paragraph()
            run = paragraph.add_run()
            run.add_text("Tools for Integrated Management of Childhood Illness")
            run.bold = True
            font = run.font
            font.size = Pt(7)
            font.color.rgb = RGBColor(61,82,122)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
            paragraph.paragraph_format.space_after = Pt(3)
            paragraph = ccell.add_paragraph()
            run = paragraph.add_run()
            pic = run.add_picture(r"C:\Users\langhe\Pictures\timci\phone.png")
            pic.width = int(pic.width/150)
            pic.height = int(pic.height/150)
            paragraph.add_run("   ____________________")
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            paragraph.paragraph_format.space_after = Pt(3)
            paragraph = ccell.add_paragraph("Date: ____ / ____ / _______")
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            
            ccell = tab.cell(2*p,1)
            paragraph = ccell.paragraphs[0]
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            run = paragraph.add_run()
            pic = run.add_picture(infile1)
            paragraph.paragraph_format.space_after = Pt(0)
            paragraph = ccell.add_paragraph(s1)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            
            # IHI header
            tab.cell(2*p-1,2).merge(tab.cell(2*p-1,3))
            ccell = tab.cell(2*p-1,2)
            paragraph = ccell.paragraphs[0]
            run = paragraph.add_run()
            pic = run.add_picture(r"C:\Users\langhe\Pictures\logo\ihi2.png")
            pic.width = int(pic.width/2)
            pic.height = int(pic.height/2)
            
            # String which represents the QR code 
            s2 = "T-F{0:04d}-P{1:04d}".format(f, n0-1+2*p)

            # Generate QR code 
            url = pyqrcode.create(s2)

            # Create and save the png file naming "myqr.png" 
            url.png(os.path.join(directory, s2+".png"), scale = 2)
            infile2 = os.path.join(os.path.join(directory, s2+".png"))
            
            ccell = tab.cell(2*p,2)
            paragraph = ccell.paragraphs[0]
            run = paragraph.add_run()
            run.add_text("TIMCI")
            run.bold = True
            font = run.font
            font.size = Pt(9)
            font.color.rgb = RGBColor(125,187,214)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
            paragraph.paragraph_format.space_after = Pt(0)
            paragraph = ccell.add_paragraph()
            run = paragraph.add_run()
            run.add_text("Tools for Integrated Management of Childhood Illness")
            run.bold = True
            font = run.font
            font.size = Pt(7)
            font.color.rgb = RGBColor(61,82,122)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
            paragraph.paragraph_format.space_after = Pt(3)
            paragraph = ccell.add_paragraph()
            run = paragraph.add_run()
            pic = run.add_picture(r"C:\Users\langhe\Pictures\timci\phone.png")
            pic.width = int(pic.width/150)
            pic.height = int(pic.height/150)
            paragraph.add_run("   ____________________")
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            paragraph.paragraph_format.space_after = Pt(3)
            paragraph = ccell.add_paragraph("Date: ____ / ____ / _______")
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            
            ccell = tab.cell(2*p,3)
            paragraph = ccell.paragraphs[0]
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            run = paragraph.add_run()
            pic = run.add_picture(infile2)
            paragraph.paragraph_format.space_after = Pt(0)
            paragraph = ccell.add_paragraph(s2)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            
        # save the doc
        doc.save(os.path.join(directory, word_doc))

### Empty card template

In [6]:
def generate_empty_card_tz_templates(directory, df, n0, n1):
    
    for _, row in df.iterrows():
        
        f = row['name']
        fname = row['label']
        
        word_doc = 'TIMCI Tanzania - F{0:04d} {1} - Card template.docx'.format(f, fname)
        
        # Create a new Word document
        doc = Document()
        doc.styles['Normal'].font.name = "Segoe UI"
        doc.styles['Normal'].font.size = Pt(10)
        tight_margin3(doc)
        
        # add a table to the end and create a reference variable
        # extra row is so we can add the header row
        tab = doc.add_table(rows=n1-n0+1, cols=4)

        # add rows
        for p in range(0, int((n1-n0+1)/2)):
            
            # IHI header
            tab.cell(2*p,0).merge(tab.cell(2*p,1))
            ccell = tab.cell(2*p,0)
            paragraph = ccell.paragraphs[0]
            run = paragraph.add_run()
            pic = run.add_picture(r"C:\Users\langhe\Pictures\logo\ihi2.png")
            pic.width = int(pic.width/2)
            pic.height = int(pic.height/2)
            
            ccell = tab.cell(2*p+1,0)
            paragraph = ccell.paragraphs[0]
            run = paragraph.add_run()
            run.add_text("TIMCI")
            run.bold = True
            font = run.font
            font.size = Pt(9)
            font.color.rgb = RGBColor(125,187,214)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
            paragraph.paragraph_format.space_after = Pt(0)
            paragraph = ccell.add_paragraph()
            run = paragraph.add_run()
            run.add_text("Tools for Integrated")
            run.add_break()
            run.add_text("Management of")
            run.add_break()
            run.add_text("Childhood Illness")
            run.bold = True
            font = run.font
            font.size = Pt(7)
            font.color.rgb = RGBColor(61,82,122)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
            paragraph.paragraph_format.space_after = Pt(3)
            paragraph = ccell.add_paragraph()
            run = paragraph.add_run()
            pic = run.add_picture(r"C:\Users\langhe\Pictures\timci\phone.png")
            pic.width = int(pic.width/150)
            pic.height = int(pic.height/150)
            paragraph.add_run("   ____________________")
            paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
            paragraph.paragraph_format.space_after = Pt(3)
            paragraph = ccell.add_paragraph("\n")
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            
            ccell = tab.cell(2*p+1,1)
            paragraph = ccell.paragraphs[0]
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            run = paragraph.add_run()
            paragraph.paragraph_format.space_after = Pt(0)
            
            # IHI header
            tab.cell(2*p,2).merge(tab.cell(2*p,3))
            ccell = tab.cell(2*p,2)
            paragraph = ccell.paragraphs[0]
            run = paragraph.add_run()
            pic = run.add_picture(r"C:\Users\langhe\Pictures\logo\ihi2.png")
            pic.width = int(pic.width/2)
            pic.height = int(pic.height/2)
            
            ccell = tab.cell(2*p+1,2)
            paragraph = ccell.paragraphs[0]
            run = paragraph.add_run()
            run.add_text("TIMCI")
            run.bold = True
            font = run.font
            font.size = Pt(9)
            font.color.rgb = RGBColor(125,187,214)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
            paragraph.paragraph_format.space_after = Pt(0)
            paragraph = ccell.add_paragraph()
            run = paragraph.add_run()
            run.add_text("Tools for Integrated")
            run.add_break()
            run.add_text("Management of Childhood\nIllness")
            run.add_break()
            run.add_text("Childhood Illness")
            run.bold = True
            font = run.font
            font.size = Pt(7)
            font.color.rgb = RGBColor(61,82,122)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
            paragraph.paragraph_format.space_after = Pt(3)
            paragraph = ccell.add_paragraph()
            run = paragraph.add_run()
            pic = run.add_picture(r"C:\Users\langhe\Pictures\timci\phone.png")
            pic.width = int(pic.width/150)
            pic.height = int(pic.height/150)
            paragraph.add_run("   ____________________")
            paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
            paragraph.paragraph_format.space_after = Pt(3)
            paragraph = ccell.add_paragraph("\n")
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            
            ccell = tab.cell(2*p+1,3)
            paragraph = ccell.paragraphs[0]
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            run = paragraph.add_run()
            paragraph.paragraph_format.space_after = Pt(0)
            
        # save the doc
        doc.save(os.path.join(directory, word_doc))

## Other template

In [7]:
def generate_tz_qr_codes_1prow(directory, df, n0, n1):
    
    for _, row in df.iterrows():
        
        f = row['name']
        fname = row['label']
        
        word_doc = 'TIMCI Tanzania - F{0:04d} {1} - Participants {2} to {3}.docx'.format(f, fname, n0, n1)
        
        # Create a new Word document
        doc = Document()
        doc.styles['Normal'].font.name = "Segoe UI"
        doc.styles['Normal'].font.size = Pt(10)
        tight_margin2(doc)
        
        # add a table to the end and create a reference variable
        # extra row is so we can add the header row
        tab = doc.add_table(rows=2*(n1-n0+1), cols=4)

        # add rows
        for p in range(n0-n0+1, n1-n0+2):
            
            # String which represents the QR code 
            s = "T-F{0:04d}-P{1:04d}".format(f, n0-1+p)

            # Generate QR code 
            url = pyqrcode.create(s)

            # Create and save the png file naming "myqr.png" 
            url.png(os.path.join(directory, s+".png"), scale = 2)
            infile = os.path.join(os.path.join(directory, s+".png"))
            
            # QR code column 1
            tab.cell(2*p-2,0).merge(tab.cell(2*p-2,1))
            ccell = tab.cell(2*p-2,0)
            paragraph = ccell.paragraphs[0]
            run = paragraph.add_run()
            pic = run.add_picture(r"C:\Users\langhe\Pictures\logo\ihi2.png")
            pic.width = int(pic.width/2)
            pic.height = int(pic.height/2)
            run = paragraph.add_run()
            run.add_text("Sticker to be given to the participant")
            run.bold = True
            font = run.font
            font.size = Pt(7)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            
            ccell = tab.cell(2*p-1,0)
            paragraph = ccell.paragraphs[0]
            run = paragraph.add_run()
            run.add_text("TIMCI")
            run.bold = True
            font = run.font
            font.size = Pt(9)
            font.color.rgb = RGBColor(125,187,214)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
            paragraph.paragraph_format.space_after = Pt(0)
            paragraph = ccell.add_paragraph()
            run = paragraph.add_run()
            run.add_text("Tools for Integrated Management of Childhood Illness")
            run.bold = True
            font = run.font
            font.size = Pt(7)
            font.color.rgb = RGBColor(61,82,122)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
            paragraph.paragraph_format.space_after = Pt(3)
            paragraph = ccell.add_paragraph()
            run = paragraph.add_run()
            pic = run.add_picture(r"C:\Users\langhe\Pictures\timci\phone.png")
            pic.width = int(pic.width/150)
            pic.height = int(pic.height/150)
            paragraph.add_run("   ____________________")
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            paragraph.paragraph_format.space_after = Pt(3)
            paragraph = ccell.add_paragraph("Date: ____ / ____ / _______")
            paragraph.add_run("\n")
            font = run.font
            font.size = Pt(5)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            
            ccell = tab.cell(2*p-1,1)
            paragraph = ccell.paragraphs[0]
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            run = paragraph.add_run()
            pic = run.add_picture(infile)
            paragraph.paragraph_format.space_after = Pt(0)
            paragraph = ccell.add_paragraph(s)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            
            # QR code column 2
            tab.cell(2*p-2,2).merge(tab.cell(2*p-2,3))
            ccell = tab.cell(2*p-2,2)
            paragraph = ccell.paragraphs[0]
            run = paragraph.add_run()
            pic = run.add_picture(r"C:\Users\langhe\Pictures\logo\ihi2.png")
            pic.width = int(pic.width/2)
            pic.height = int(pic.height/2)
            run = paragraph.add_run()
            run.add_text("Sticker to be placed on the signed ICF kept in the site study file")
            run.bold = True
            font = run.font
            font.size = Pt(7)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            
            ccell = tab.cell(2*p-1,2)
            paragraph = ccell.paragraphs[0]
            run = paragraph.add_run()
            run.add_text("TIMCI")
            run.bold = True
            font = run.font
            font.size = Pt(9)
            font.color.rgb = RGBColor(125,187,214)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
            paragraph.paragraph_format.space_after = Pt(0)
            paragraph = ccell.add_paragraph()
            run = paragraph.add_run()
            run.add_text("Tools for Integrated Management of Childhood Illness")
            run.bold = True
            font = run.font
            font.size = Pt(7)
            font.color.rgb = RGBColor(61,82,122)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT
            paragraph.paragraph_format.space_after = Pt(3)
            paragraph = ccell.add_paragraph()
            run = paragraph.add_run()
            pic = run.add_picture(r"C:\Users\langhe\Pictures\timci\phone.png")
            pic.width = int(pic.width/150)
            pic.height = int(pic.height/150)
            paragraph.add_run("   ____________________")
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            paragraph.paragraph_format.space_after = Pt(3)
            paragraph = ccell.add_paragraph("Date: ____ / ____ / _______")
            paragraph.add_run("\n")
            font = run.font
            font.size = Pt(5)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            
            ccell = tab.cell(2*p-1,3)
            paragraph = ccell.paragraphs[0]
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            run = paragraph.add_run()
            pic = run.add_picture(infile)
            paragraph.paragraph_format.space_after = Pt(0)
            paragraph = ccell.add_paragraph(s)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            
        # save the doc
        doc.save(os.path.join(directory, word_doc))

## List of facilities

In [11]:
import pandas as pd
df = pd.read_csv(r"C:\Users\langhe\switchdrive\Institution\TIMCI\1 Project details\11 DM\3 ODK\odk_forms\06 Tanzania\1-RCT-LS\01a-TIMCI-CRF-Facility-ref-form\facilities.csv", sep=",")
df['name'] = df.apply(lambda row: int(row['name'].replace('F','')), axis=1)

pilot_df = df[df['pilot']==1]
nonpilot_df = df[df['pilot']==0]
spa_df = df[df['spa']==1]
nonpilot_df

,name,label,type,category,ccode,lvl2,village_street,ward,intervention,spa,Latitude,Longitude,hf_number,pilot
1,1,Igombe 56,dispensary,rural,T,Kaliua,Imara,Igombe Mkulu,1,0,-4.544040,32.241600,101565-0,0
3,9,Igwisi,dispensary,rural,T,Kaliua,Igwisi,Igwisi,0,0,-4.918523,31.900047,101600-5,0
4,11,Kaliua govn,health center,urban,T,Kaliua,Kaliua Magharibi,Kaliua,0,0,-5.043827,31.804701,109992-8,0
5,5,Kasungu,dispensary,rural,T,Kaliua,Kasungu,Kaliua,0,0,-5.011027,31.788824,112467-6,0
6,14,Kazaroho,health center,rural,T,Kaliua,Kazaroho,Kazaroho,0,1,-4.988730,31.853500,102518-8,0
7,84,Kilimawe,dispensary,rural,T,Kaliua,Songambele,Uyowa,2,0,-4.591180,32.021150,102849-7,0
8,3,Limbula,dispensary,rural,T,Kaliua,Limbula,Ugunga,1,0,-5.124160,31.740900,103532-8,0
9,80,Makingi,dispensary,rural,T,Kaliua,Makingi,Ichemba,2,0,-4.682890,32.290270,111210-1,0
10,7,Nyasa,dispensary,rural,T,Kaliua,Nyasa,Seleli,0,0,-4.326170,32.261620,106814-7,0
11,12,Sasu,dispensary,rural,T,Kaliua,Sasu,Sasu,1,1,NaN,NaN,107307-1,0


## Generate QR codes

In [ ]:
#generate_tz_qr_codes(r'QR_codes_Tanzania', pilot_df, 1, 100)

In [12]:
generate_qr_codes_2prow(r'QR_codes_Tanzania', nonpilot_df, 3501, 3506)

In [ ]:
generate_qr_codes_2prow(r'QR_codes_Tanzania', pilot_df, 3001, 3500)

In [ ]:
#generate_empty_card_tz_templates(r'QR_codes_Tanzania', pilot_df, 1, 10)

### Healtcare provider codes

In [ ]:
def generate_hcpqr_codes_2prow(directory, df, n):
    
    for _, row in df.iterrows():
        
        f = row['name']
        fname = row['label']
        
        word_doc = 'TIMCI Tanzania - F{0:04d} {1} - Providers 1 to {2}.docx'.format(f, fname, n)
        
        # Create a new Word document
        doc = Document()
        doc.styles['Normal'].font.name = "Segoe UI"
        doc.styles['Normal'].font.size = Pt(10)
        tight_margin(doc)
        
        # add a table to the end and create a reference variable
        # extra row is so we can add the header row
        tab = doc.add_table(rows=n, cols=2)
        
        # add rows
        for p in range(1, n):

            # String which represents the QR code 
            s1 = "T-F{0:04d}-H{1:02d}".format(f, p)

            # Generate QR code 
            url = pyqrcode.create(s1)

            # Create and save the png file naming "myqr.png" 
            url.png(os.path.join(directory, s1+".png"), scale = 2)
            
            infile1 = os.path.join(os.path.join(directory, s1+".png"))
            
            paragraph = tab.cell(p,0).paragraphs[0]
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            run = paragraph.add_run()
            pic = run.add_picture(infile1)
            paragraph.paragraph_format.space_after = Pt(0)
            paragraph = tab.cell(p,0).add_paragraph(s1)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            paragraph.paragraph_format.space_after = Pt(3)
            paragraph = tab.cell(p,0).add_paragraph("Date: ____ / ____ / _______")
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            
            paragraph = tab.cell(p,1).paragraphs[0]
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            paragraph = tab.cell(p,1).add_paragraph("Quarter of enrolment: _________________________")
            paragraph = tab.cell(p,1).add_paragraph("First name: ___________________________________")
            paragraph = tab.cell(p,1).add_paragraph("Last name: ____________________________________")
            paragraph = tab.cell(p,1).add_paragraph("Position: _____________________________________")
            
        # save the doc
        doc.save(os.path.join(directory, word_doc))

In [ ]:
#generate_hcpqr_codes_2prow(r'QR_codes_Tanzania', pilot_df, 20)

In [ ]:
generate_hcpqr_codes_2prow(r'QR_codes_Tanzania', spa_df, 20)